In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission_v2.csv
test-jpg-v2
test-tif-v3
test_v2_file_mapping.csv
train-jpg
train-tif-v2
train_v2.csv



In [2]:
train = pd.read_csv('../input/train_v2.csv')
test = pd.read_csv('../input/sample_submission_v2.csv')

In [3]:
labels = train['tags'].apply(lambda x: x.split(' '))

In [4]:
label_list=[]
for i in labels:
    for j in i:
        if j not in label_list:
            label_list.append(j)

In [5]:
x_train = []
x_test = []
y_train = []

In [6]:
label_map = {l: i for i, l in enumerate(label_list)}

In [7]:
label_map

{'agriculture': 2,
 'artisinal_mine': 13,
 'bare_ground': 12,
 'blooming': 14,
 'blow_down': 16,
 'clear': 3,
 'cloudy': 9,
 'conventional_mine': 11,
 'cultivation': 7,
 'habitation': 5,
 'haze': 0,
 'partly_cloudy': 10,
 'primary': 1,
 'road': 6,
 'selective_logging': 15,
 'slash_burn': 8,
 'water': 4}

In [8]:
from tqdm import tqdm
import cv2

In [9]:
for f, tags in tqdm(train.values, miniters=1000):
    img = cv2.imread('../input/train-jpg/{}.jpg'.format(f))
    img = cv2.resize(img,(64,64))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_train.append(img)
    y_train.append(targets)

100%|██████████| 40479/40479 [01:39<00:00, 405.39it/s]


In [10]:
for f, tags in tqdm(test.values, miniters=1000):
    img = cv2.imread('../input/test-jpg-v2/{}.jpg'.format(f))
    img = cv2.resize(img,(64,64))
    x_test.append(cv2.resize(img,(64, 64)))

100%|██████████| 61191/61191 [02:19<00:00, 439.15it/s]


In [11]:
y_train = np.array(y_train)
x_train = np.array(x_train, np.float32)/255.0
x_test  = np.array(x_test, np.float32)/255.0

In [12]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(40479, 64, 64, 3)
(61191, 64, 64, 3)
(40479, 17)


In [13]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
np.random.seed(1671)

Using TensorFlow backend.


In [14]:
model = Sequential()
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(64, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model.add(Conv2D(128, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(17))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 64)        0         
__________

In [15]:
OPTIMIZER = Adam( lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=OPTIMIZER , metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=1, validation_split=0.25)

Train on 30359 samples, validate on 10120 samples
Epoch 1/2
30359/30359 [==============================] - 961s - loss: 0.3709 - acc: 0.8315 - val_loss: 0.3629 - val_acc: 0.8311
Epoch 2/2
30359/30359 [==============================] - 739s - loss: 0.3632 - acc: 0.8311 - val_loss: 0.3626 - val_acc: 0.8311


In [16]:
OPTIMIZER.lr = 0.001
history = model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=1, validation_split=0.25)

Train on 30359 samples, validate on 10120 samples
Epoch 1/2
30359/30359 [==============================] - 722s - loss: 0.3629 - acc: 0.8311 - val_loss: 0.3627 - val_acc: 0.8311
Epoch 2/2
30359/30359 [==============================] - 935s - loss: 0.3628 - acc: 0.8311 - val_loss: 0.3628 - val_acc: 0.8311


In [19]:
y_test=[]
p_test = model.predict(x_test)
y_test.append(p_test)

In [29]:
result = np.array(y_test[0])
result = pd.DataFrame(result, columns = label_list)
result

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
1,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
2,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
3,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
4,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
5,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
6,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
7,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
8,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406
9,0.022776,0.365496,0.094962,0.237375,0.062878,0.028744,0.060762,0.033242,0.001901,0.015857,0.056723,0.001104,0.006948,0.003322,0.003164,0.003339,0.001406


In [35]:
from tqdm import tqdm
thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))
    
test['tags'] = preds
test.to_csv('submission.csv', index=False)

100%|██████████| 61191/61191 [02:25<00:00, 420.03it/s]


In [39]:
submission.csv

NameError: name 'submission' is not defined